# Project 01 - Color Compression

## Thông tin sinh viên

- Họ và tên: Nguyễn Thế Hiển
- MSSV:22127107 
- Lớp:22CLC08

## Import các thư viện liên quan

In [1]:
from PIL import Image
import numpy as np
import matplotlib.pyplot as plt

## Helper functions

In [2]:
def read_img(img_path):
    '''
    Read image from img_path

    Parameters
    ----------
    img_path : str
        Path of image

    Returns
    -------
        Image (2D)
    '''

    # YOUR CODE HERE
    image = Image.open(img_path)
    image_2d = np.array(image)
    return image_2d


def show_img(img_2d):
    '''
    Show image

    Parameters
    ----------
    img_2d : <your type>
        Image (2D)
    '''

    # YOUR CODE HERE
    plt.imshow(img_2d)
    plt.axis('off')
    plt.show()


def save_img(img_2d, img_path):
    '''
    Save image to img_path

    Parameters
    ----------
    img_2d : <your type>
        Image (2D)
    img_path : str
        Path of image
    '''

    # YOUR CODE HERE
    image = Image.fromarray(img_2d.astype('uint8'))
    image.save(img_path)


def convert_img_to_1d(img_2d):
    '''
    Convert 2D image to 1D image

    Parameters
    ----------
    img_2d : <your type>
        Image (2D)

    Returns
    -------
        Image (1D)
    '''

    # YOUR CODE HERE
    return img_2d.reshape((-1, img_2d.shape[2]))

def kmeans(img_1d, k_clusters, max_iter, init_centroids='random'):
    '''
    K-Means algorithm

    Parameters
    ----------
    img_1d : np.ndarray with shape=(height * width, num_channels)
        Original (1D) image
    k_clusters : int
        Number of clusters
    max_iter : int
        Max iterator
    init_centroids : str, default='random'
        The method used to initialize the centroids for K-means clustering
        'random' --> Centroids are initialized with random values between 0 and 255 for each channel
        'in_pixels' --> A random pixel from the original image is selected as a centroid for each cluster

    Returns
    -------
    centroids : np.ndarray with shape=(k_clusters, num_channels)
        Stores the color centroids for each cluster
    labels : np.ndarray with shape=(height * width, )
        Stores the cluster label for each pixel in the image
    '''
    
    # YOUR CODE HERE
    np.random.seed(42)

    if init_centroids == 'random':
        centroids = np.random.randint(0, 256, size=(k_clusters, img_1d.shape[1]))
    elif init_centroids == 'in_pixels':
        centroids = img_1d[np.random.choice(img_1d.shape[0], k_clusters, replace=False)]
    else:
        raise ValueError("init_centroids phải là 'random' hoặc 'in_pixels'")

    for _ in range(max_iter):
        distances = np.linalg.norm(img_1d[:, np.newaxis] - centroids, axis=2)
        labels = np.argmin(distances, axis=1)

        new_centroids = np.array([img_1d[labels == i].mean(axis=0) if np.any(labels == i) else centroids[i] 
                                  for i in range(k_clusters)])
        if np.all(centroids == new_centroids):
            break

        centroids = new_centroids

    return centroids, labels


def generate_2d_img(img_2d_shape, centroids, labels):
    '''
    Generate a 2D image based on K-means cluster centroids

    Parameters
    ----------
    img_2d_shape : tuple (height, width, 3)
        Shape of image
    centroids : np.ndarray with shape=(k_clusters, num_channels)
        Store color centroids
    labels : np.ndarray with shape=(height * width, )
        Store label for pixels (cluster's index on which the pixel belongs)

    Returns
    -------
        New image (2D)
    '''

    # YOUR CODE HERE
    new_img_1d = centroids[labels].astype(np.uint8)
    # 1D (height * width, channels) -> 2D (height, width, channels)
    new_img_2d = new_img_1d.reshape(img_2d_shape)
    return new_img_2d


# Your additional functions here


## Your tests

In [3]:
# YOUR CODE HERE
def test_functions():
    img_path = "path/to/your/test/image.jpg"

    img_2d = read_img(img_path)
    assert img_2d is not None, "Failed to read image"
    assert len(img_2d.shape) == 3 and img_2d.shape[2] == 3, "Image is not a valid RGB image"

    print("Displaying original image:")
    show_img(img_2d)

    img_1d = convert_img_to_1d(img_2d)
    assert img_1d.shape[0] == img_2d.shape[0] * img_2d.shape[1], "Conversion to 1D failed"
    assert img_1d.shape[1] == 3, "Conversion to 1D failed: number of channels should be 3"

    k_clusters = 5
    max_iter = 100
    centroids_random, labels_random = kmeans(img_1d, k_clusters, max_iter, init_centroids='random')
    assert centroids_random.shape == (k_clusters, img_1d.shape[1]), "K-Means centroids shape mismatch for 'random'"
    assert labels_random.shape == (img_1d.shape[0],), "K-Means labels shape mismatch for 'random'"

    new_img_2d_random = generate_2d_img(img_2d.shape, centroids_random, labels_random)
    assert new_img_2d_random.shape == img_2d.shape, "Generated image shape mismatch for 'random'"

    print("Displaying new image with 'random' initialization:")
    show_img(new_img_2d_random)

    save_img(new_img_2d_random, "test_image_result_random.png")
    save_img(new_img_2d_random, "test_image_result_random.pdf")

    centroids_in_pixels, labels_in_pixels = kmeans(img_1d, k_clusters, max_iter, init_centroids='in_pixels')
    assert centroids_in_pixels.shape == (k_clusters, img_1d.shape[1]), "K-Means centroids shape mismatch for 'in_pixels'"
    assert labels_in_pixels.shape == (img_1d.shape[0],), "K-Means labels shape mismatch for 'in_pixels'"

    new_img_2d_in_pixels = generate_2d_img(img_2d.shape, centroids_in_pixels, labels_in_pixels)
    assert new_img_2d_in_pixels.shape == img_2d.shape, "Generated image shape mismatch for 'in_pixels'"

    print("Displaying new image with 'in_pixels' initialization:")
    show_img(new_img_2d_in_pixels)

    save_img(new_img_2d_in_pixels, "test_image_result_in_pixels.png")
    save_img(new_img_2d_in_pixels, "test_image_result_in_pixels.pdf")

    print("All tests passed.")

## Main FUNCTION

In [4]:
# YOUR CODE HERE
def main():
    img_path = input("Enter the path to the image: ")
    k_clusters = int(input("Enter the number of clusters: "))
    max_iter = int(input("Enter the maximum number of iterations: "))
    save_path_base = input("Enter the base path to save the new image (without extension): ")

    img_2d = read_img(img_path)
    img_1d = convert_img_to_1d(img_2d)

    print("Original Image:")
    show_img(img_2d)

    # Run K-means with 'random' initialization
    centroids_random, labels_random = kmeans(img_1d, k_clusters, max_iter, init_centroids='random')
    new_img_2d_random = generate_2d_img(img_2d.shape, centroids_random, labels_random)

    print("New Image with 'random' initialization:")
    show_img(new_img_2d_random)

    save_path_random_png = save_path_base + "_random.png"
    save_path_random_pdf = save_path_base + "_random.pdf"
    save_img(new_img_2d_random, save_path_random_png)
    save_img(new_img_2d_random, save_path_random_pdf)

    print(f"Image with 'random' initialization saved as {save_path_random_png} and {save_path_random_pdf}")

    # Run K-means with 'in_pixels' initialization
    centroids_in_pixels, labels_in_pixels = kmeans(img_1d, k_clusters, max_iter, init_centroids='in_pixels')
    new_img_2d_in_pixels = generate_2d_img(img_2d.shape, centroids_in_pixels, labels_in_pixels)

    print("New Image with 'in_pixels' initialization:")
    show_img(new_img_2d_in_pixels)

    save_path_in_pixels_png = save_path_base + "_in_pixels.png"
    save_path_in_pixels_pdf = save_path_base + "_in_pixels.pdf"
    save_img(new_img_2d_in_pixels, save_path_in_pixels_png)
    save_img(new_img_2d_in_pixels, save_path_in_pixels_pdf)

    print(f"Image with 'in_pixels' initialization saved as {save_path_in_pixels_png} and {save_path_in_pixels_pdf}")


In [5]:
# Call main function
main() 